In [24]:
# Install required packages
!pip install crewai crewai-tools gradio openai arxiv scholarly beautifulsoup4 requests

import os
import gradio as gr
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool
from langchain_openai import ChatOpenAI
import arxiv
import scholarly
from typing import List, Dict
import json

/opt/anaconda3/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=89450) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


In [25]:
# Initialize the LLM
llm = ChatOpenAI(
    model="gpt-4",
    temperature=0.7
)

In [26]:
# Define custom tools for research
class ArxivSearchTool:
    def search_papers(self, query: str, max_results: int = 5) -> List[Dict]:
        """Search for papers on arXiv"""
        search = arxiv.Search(
            query=query,
            max_results=max_results,
            sort_by=arxiv.SortCriterion.Relevance
        )

        papers = []
        for result in search.results():
            papers.append({
                "title": result.title,
                "authors": [author.name for author in result.authors],
                "abstract": result.summary,
                "url": result.entry_id,
                "published": str(result.published)
            })
        return papers

In [27]:
# Initialize tools
arxiv_tool = ArxivSearchTool()

In [28]:
# Define Agents
topic_explainer = Agent(
    role='Topic Explainer',
    goal='Explain complex research topics in clear, accessible language and provide comprehensive overviews',
    backstory="""You are an expert educator with deep knowledge across multiple research domains.
    You excel at breaking down complex concepts into understandable explanations while maintaining accuracy.
    You understand the importance of providing context and connecting ideas to help researchers grasp new topics quickly.""",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

literature_finder = Agent(
    role='Literature Finder',
    goal='Find and summarize relevant research papers, identifying key contributions and methodologies',
    backstory="""You are a seasoned research librarian with expertise in academic literature search.
    You know how to identify the most relevant and impactful papers in any field. You're skilled at
    summarizing papers concisely while capturing their key contributions, methodologies, and findings.""",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

gap_analyzer = Agent(
    role='Gap Analyzer',
    goal='Identify research gaps, unexplored areas, and suggest promising research directions',
    backstory="""You are a research strategist with years of experience in identifying opportunities
    in academic research. You excel at spotting patterns, finding gaps in existing literature,
    and proposing innovative research directions that could advance the field.""",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

In [29]:
def create_tasks(research_topic: str):
    explain_topic_task = Task(
        description=f"""Provide a comprehensive explanation of the research topic: {research_topic}
        Include:
        1. Core concepts and definitions
        2. Historical context and evolution
        3. Current state of the field
        4. Key challenges and open questions
        5. Interdisciplinary connections""",
        agent=topic_explainer,
        expected_output="A detailed explanation of the research topic with all requested components"
    )

    find_literature_task = Task(
        description=f"""Search and analyze literature related to: {research_topic}
        For each paper found:
        1. Provide title, authors, and publication year
        2. Summarize the main contributions
        3. Explain the methodology used
        4. Highlight key findings
        5. Note limitations mentioned by authors
        Use the arxiv_tool to search for papers.""",
        agent=literature_finder,
        expected_output="A comprehensive literature review with summaries of relevant papers"
    )

    analyze_gaps_task = Task(
        description=f"""Based on the topic explanation and literature review for {research_topic}, identify:
        1. Research gaps in the current literature
        2. Unexplored or underexplored areas
        3. Methodological limitations in existing work
        4. Potential interdisciplinary opportunities
        5. Suggest 3-5 specific research questions that could address these gaps
        6. Provide a research outline for the most promising direction""",
        agent=gap_analyzer,
        expected_output="A detailed gap analysis with specific research suggestions and an outline"
    )

    return [explain_topic_task, find_literature_task, analyze_gaps_task]


In [30]:
def create_research_crew(research_topic: str):
    tasks = create_tasks(research_topic)

    crew = Crew(
        agents=[topic_explainer, literature_finder, gap_analyzer],
        tasks=tasks,
        process=Process.sequential,
        verbose=True
    )

    return crew


In [31]:
def run_research_companion(research_topic: str, specific_focus: str = None):
    """Run the research companion for a given topic"""

In [32]:
def run_research_companion(research_topic: str, specific_focus: str = None):
    """Run the research companion for a given topic"""
    # Modify topic if specific focus is provided
    if specific_focus and specific_focus.strip():
        full_topic = f"{research_topic} with focus on {specific_focus}"
    else:
        full_topic = research_topic

In [33]:
def gradio_interface(research_topic, specific_focus, api_key):
    """Gradio interface function"""

    # Update API key if provided
    if api_key and api_key.strip():
        os.environ["OPENAI_API_KEY"] = api_key

    if not research_topic:
        return "Please enter a research topic.", "", "", ""

    # Run the research companion
    results = run_research_companion(research_topic, specific_focus)
    if "error" in results:
        error_msg = f"Error: {results['error']}"
        return error_msg, error_msg, error_msg, error_msg

    # Format outputs
    explanation = results.get("explanation", results.get("full_output", ""))
    literature = results.get("literature_review", "")
    gaps = results.get("gap_analysis", "")

    # Create formatted output
    formatted_output = f"""
# Research Analysis for: {results['topic']}

## Topic Explanation
{explanation}

## Literature Review
{literature}

## Gap Analysis and Research Suggestions
{gaps}
"""

    return formatted_output, explanation, literature, gaps

# Create Gradio UI
with gr.Blocks(title="AI Research Paper Companion") as demo:
    gr.Markdown("""
    # 🔬 AI Research Paper Companion

    This tool uses AI agents to help you understand research topics, find relevant literature, and identify research gaps.

    **Features:**
    - **Topic Explainer**: Provides comprehensive explanations of research topics
    - **Literature Finder**: Searches and summarizes relevant papers
    - **Gap Analyzer**: Identifies research gaps and suggests new directions
    """)

    # Continue with the rest of your Gradio interface code...


In [34]:
def run_research_companion(research_topic, specific_focus=None):
    # Combine topic and focus
    full_topic = research_topic
    if specific_focus:
        full_topic += f" with focus on {specific_focus}"

    # Create and run the crew
    crew = create_research_crew(full_topic)

    try:
        # Execute the crew
        result = crew.kickoff()

        # Parse results
        output = {
            "topic": full_topic,
            "explanation": "",
            "literature_review": "",
            "gap_analysis": "",
            "full_output": str(result)
        }

        # Extract individual agent outputs if available
        if hasattr(result, 'tasks_output'):
            if len(result.tasks_output) >= 3:
                output["explanation"] = str(result.tasks_output[0])
                output["literature_review"] = str(result.tasks_output[1])
                output["gap_analysis"] = str(result.tasks_output[2])

        return output

    except Exception as e:
        return {
            "error": f"An error occurred: {str(e)}",
            "topic": full_topic
        }

# Gradio Interface
def gradio_interface(research_topic, specific_focus, api_key):
    """Gradio interface function"""

    # Update API key if provided
    if api_key and api_key.strip():
        os.environ["OPENAI_API_KEY"] = api_key

    if not research_topic:
        return "Please enter a research topic.", "", "", ""

    # Run the research companion
    results = run_research_companion(research_topic, specific_focus)
    if "error" in results:
        return f"Error: {results['error']}", "", "", ""

    # Format outputs
    explanation = results.get("explanation", results.get("full_output", ""))
    literature = results.get("literature_review", "")
    gaps = results.get("gap_analysis", "")

    return explanation, literature, gaps, results.get("full_output", "")


In [ ]:
import os
# Proxy fix
os.environ["NO_PROXY"] = "localhost,127.0.0.1,::1"# Complete updated interface
def gradio_interface(research_topic, specific_focus, api_key):
    if api_key and api_key.strip():
        os.environ["OPENAI_API_KEY"] = api_key
    
    if not research_topic:
        return "Please enter a research topic.", "", "", ""
    
    if not os.getenv("OPENAI_API_KEY"):
        return "Please provide your OpenAI API key.", "", "", ""
    
    results = run_research_companion(research_topic, specific_focus)
    
    if "error" in results:
        error_msg = f"Error: {results['error']}"
        return error_msg, error_msg, error_msg, error_msg
    
    explanation = results.get("explanation", "No explanation generated")
    literature = results.get("literature_review", "No literature review generated") 
    gaps = results.get("gap_analysis", "No gap analysis generated")
    
    full_report = f"""# 🔬 Research Analysis: {results.get('topic', research_topic)}

## 📚 Topic Explanation
{explanation}

## 🔍 Literature Review  
{literature}

## 🎯 Gap Analysis & Research Suggestions
{gaps}
"""
    
    return explanation, literature, gaps, full_report

# Updated Gradio Interface
with gr.Blocks(title="AI Research Paper Companion", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🔬 AI Research Paper Companion
    
    **Powered by AI Agents:** Topic Explainer | Literature Finder | Gap Analyzer
    """)
    
    with gr.Row():
        with gr.Column(scale=2):
            api_key = gr.Textbox(label="🔑 OpenAI API Key", type="password")
            topic = gr.Textbox(label="🎯 Research Topic", placeholder="e.g., Machine Learning, Quantum Computing")
            focus = gr.Textbox(label="🔍 Specific Focus (Optional)", placeholder="e.g., Reinforcement Learning")
            submit = gr.Button("🚀 Start Analysis", variant="primary", size="lg")
    
    with gr.Tab("📊 Agent Results"):
        with gr.Row():
            with gr.Column():
                gr.Markdown("### 📚 Topic Explainer")
                topic_output = gr.Textbox(label="", lines=12, show_label=False)
            
            with gr.Column():
                gr.Markdown("### 🔍 Literature Finder") 
                literature_output = gr.Textbox(label="", lines=12, show_label=False)
            
            with gr.Column():
                gr.Markdown("### 🎯 Gap Analyzer")
                gap_output = gr.Textbox(label="", lines=12, show_label=False)
    
    with gr.Tab("📋 Complete Report"):
        full_output = gr.Markdown("")
    
    submit.click(
        fn=gradio_interface,
        inputs=[topic, focus, api_key],
        outputs=[topic_output, literature_output, gap_output, full_output]
    )

# Sabse easy - port parameter hi remove kar do
demo.launch(
    share=True,
    server_name="0.0.0.0", 
    # server_port=7860,  # Ye line comment kar do
    inbrowser=True,
    debug=True
)

